In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
verify_few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/Food-verify.expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner-qa'
model_name = "gpt-3.5-turbo"
label_column = 'Food_answer'
text_column = 'context'
explanation_column = 'Two_step_Food_explanation'
example_selection_label_column = 'has_Food'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'label_diversity_similarity'
verify_few_shot_selection = "semantic_similarity"
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
verify=True
label_version='v1'
task_version='v1'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from question_answering_new import NewQuestionAnsweringTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
from prompt_template import load_ner_task_prompt
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.QUESTION_ANSWERING] = NewQuestionAnsweringTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
label_type = label_column.split('_')[0]
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name.split('/')[1]}"

In [9]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_ner_task_prompt(label_type, label_symbol, label_version, task_version, True)

In [10]:
test_data = pd.read_csv(data_path)

In [11]:
if use_current_explanation and not use_ground_explanation:
    explanation_column = explanation_column + "_prediction"

In [12]:
config = {
    "task_name": task_name,
    "task_type": "question_answering",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "explanation_column": explanation_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": output_guideline,
        "labels": [
            "Food",
            "Symptom",
            "Location",
            "Keyword"
        ],
        "example_selection_labels":[
            "yes",
            "no"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: Let's think step by step.\n{{{explanation_column}}}\n{{{label_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [13]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-26 03:12:43 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
if use_current_explanation:
    ds = AutolabelDataset(test_data[[text_column, explanation_column, example_selection_label_column, label_column]], config=config)
else:
    ds = AutolabelDataset(test_data[[text_column, example_selection_label_column, label_column]], config=config)

In [15]:
agent.plan(ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-26 03:12:46 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:50 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:54 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-26 03:12:56 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:56 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:56 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-26 03:12:58 httpx INFO: HTTP Request: 

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0973 │
│ Number of Examples       │ 10      │
│ Average cost per example │ $0.0097 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting Food entities that are related to foodborne illness incident from text. In the 
given text, your task is to label Food entities that are specific food item that caused the potential foodborne 
illness incident. Note that you should surround the extracted entities in the text with "^^" and "^^". Use the 
following examples as a guide for your predictions and format your responses similarly.

Your answer will consist of an explanation, followed by the correct labeled sentence. The last line of the answer 
should always be is JSON format with one key: {"label": "the correct labeled sentence"}.
Note that if the given text does not include any Food entity related to foodborne illness, your labeled sentence 
will be the input text.

Some examples with their output answers are provided below:

Input: I woke up with either a stomach bug or food poisoning , and can't get out of bed 😔 I have four papers to 
write , a final , and four performances this week . Please pray I'm better ASAP 😥
Output: Let's think step by step.
The text indicates a foodborne illness incident. However, no Food entity in the text is relevant to foodborne 
illness incident.
{"label": "I woke up with either a stomach bug or food poisoning , and can't get out of bed \ud83d\ude14 I have 
four papers to write , a final , and four performances this week . Please pray I'm better ASAP \ud83d\ude25"}

Input: All that free food yesterday made me SO sick [ EMOJI_face_with_tears_of_joy ] idk if I got a bit of food 
poisoning or if a random 24 hour bug suddenly came on , but WOW it was intense !
Output: Let's think step by step.
The text indicates a foodborne illness incident. The word "free food" in the text is a Food entity, which is a 
specific food item that caused the potential foodborne illness incident.
{"label": "All that ^^free food^^ yesterday made me SO sick [ EMOJI_face_with_tears_of_joy ] idk if I got a bit of 
food poisoning or if a random 24 hour bug suddenly came on , but WOW it was intense !"}

Input: I should be enroute fm ATL to ORF now , but had to change travel plans . . . been dealing with food 
poisoning the past 15 hours . Ugh , miserable .
Output: Let's think step by step.
The text indicates a foodborne illness incident. However, no Food entity in the text is relevant to foodborne 
illness incident.
{"label": "I should be enroute fm ATL to ORF now , but had to change travel plans . . . been dealing with food 
poisoning the past 15 hours . Ugh , miserable ."}

Input: I am either dying or have food poisoning . . . I'm going with food poisoning since I cooked meat today [ 
EMOJI_face_with_medical_mask ]
Output: Let's think step by step.
The text indicates a foodborne illness incident. The word "meat" in the text is a Food entity, which is a specific 
food item that caused the potential foodborne illness incident.
{"label": "I am either dying or have food poisoning . . . I'm going with food poisoning since I cooked ^^meat^^ 
today [ EMOJI_face_with_medical_mask ]"}

Input: @USER @USER @USER LOL DONT GET FOOD POISONING ! Me and my friend both got a taco in a bag on Tuesday and we 
were both extremely sick the next morning
Output: Let's think step by step.
The text indicates a foodborne illness incident. The word "taco" in the text is a Food entity, which is a specific 
food item that caused the potential foodborne illness incident.
{"label": "@USER @USER @USER LOL DONT GET FOOD POISONING ! Me and my friend both got a ^^taco^^ in a bag on Tuesday
and we were both extremely sick the next morning"}

Input: @USER LOL I know ! I've had a stomach bug or food poisoning since yesterday afternoon
Output: Let's think step by step.
The text indicates a foodborne illness incident. However, no Food entity in the text is relevant to foodborne 
illness incident.
{"label": "@USER LOL I know ! I've had a stomach bug or food poisoning since yesterday afternoon"}

Input: @USER Tell them you got diarrhea and can't leave the toilet lmfao [ EMOJ

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [16]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

2024-03-20 16:09:46 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 16:09:47 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 16:09:48 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 16:09:49 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 16:09:49 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 16:09:51 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 16:09:51 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 16:09:52 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 16:09:52 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 16:09:53 ht

Actual Cost: 0.0098

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ f1     ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.3      │ 10      │ 1.0             │ 0.9264 │
└──────────┴─────────┴─────────────────┴────────┘

In [17]:
metrics = ds.eval()

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ f1     ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.3      │ 10      │ 1.0             │ 0.9264 │
└──────────┴─────────┴─────────────────┴────────┘

In [18]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))

In [19]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []
    
if not verify:
    raise StopExecution

# Verification Stage

In [20]:
from utils import get_predictions, check_prediction, get_verify_df, construct_results

In [21]:
verify_question_column = f'{label_type}_verify_question'
verify_answer_column = f'{label_type}_verify_answer'
pos_column = f'{label_column}_pos'

In [22]:
test_data[[verify_question_column, pos_column]] = ds.df[[f'{label_column}_label']].apply(lambda x: get_predictions(x[f'{label_column}_label'], label_symbol), axis=1,result_type="expand")

In [23]:
test_data[verify_answer_column] = test_data[['CategorizedLabels', verify_question_column]].apply(lambda x: check_prediction(x['CategorizedLabels'], x[verify_question_column], label_type), axis=1)

In [24]:
new_test_data = get_verify_df(test_data, verify_question_column, verify_answer_column, text_column, pos_column)

In [25]:
verify_task_guideline = (f"You are an expert at identifying {label_type} entities that are related to foodborne illness incident from text. " 
                         f"In the given text, your task is to verify if a given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text. "
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the word "{{{verify_question_column}}}" in the given text is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')}?'''

In [26]:
output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_verify_' + os.path.split(data_path)[-1]
output_verify_path = os.path.join(output_dir, output_verify_name)
output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_final_' + os.path.split(data_path)[-1]
output_final_path = os.path.join(output_dir, output_final_name)

In [27]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": em_provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": f'''You will answer with just the correct output label ("Yes" or "No") and nothing else.''',
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": verify_few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: {{{verify_answer_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [28]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-14 22:47:45 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.


In [29]:
verify_ds = AutolabelDataset(new_test_data.drop(['text_idx',pos_column], axis=1), config=config)

In [30]:
agent.plan(verify_ds)

2024-03-14 22:47:47 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:51 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:55 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-14 22:47:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:58 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0371 │
│ Number of Examples       │ 4       │
│ Average cost per example │ $0.0093 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying Food entities that are related to foodborne illness incident from text. In the 
given text, your task is to verify if a given word is a Food entity that is specific food item that caused the 
potential foodborne illness incident. in the given text. Use the following examples as a guide for your analysis 
and format your responses similarly.

You will answer with just the correct output label ("Yes" or "No") and nothing else.

Some examples with their output answers are provided below:

Context: ok i lied abt having food poisoning so I didn't have to go into work yesterday but now that I do have to 
go to work today I feel like shit wtffff
Question: Do you think the word to go in the given text is a Food entity that is specific food item that caused the
potential foodborne illness incident.?
Answer: No
Context: Last night I got food poisoning so I couldn't have my coffee this morning and I had to go into work 
anyways so this day has been terrible for everyone involved
Question: Do you think the word this day has in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: No
Context: So tonight I think I suffered from food poisoning & was in my bed sick , then some of my pals come back 
and i have never seen people in a . .
Question: Do you think the word think I suffered from in the given text is a Food entity that is specific food item
that caused the potential foodborne illness incident.?
Answer: No
Context: Think I might have gotten a mild case of food-poisoning today - so that's been super fun .
Question: Do you think the word gotten in the given text is a Food entity that is specific food item that caused 
the potential foodborne illness incident.?
Answer: No
Context: I've had a bad food poisoning experience le curry pot . [ EMOJI_loudly_crying_face ] [ 
EMOJI_loudly_crying_face ] I'm scarred .
Question: Do you think the word le curry pot in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: Yes
Context: I am either dying or have food poisoning . . . I'm going with food poisoning since I cooked meat today [ 
EMOJI_face_with_medical_mask ]
Question: Do you think the word meat in the given text is a Food entity that is specific food item that caused the 
potential foodborne illness incident.?
Answer: Yes
Context: @USER " Rambling Zuma " - a superb name for a food poisoning symptom . " I can't come to work today boss -
been up all night with the Rambling Zuma . Must have been a dodgy kebab " .
Question: Do you think the word kebab in the given text is a Food entity that is specific food item that caused the
potential foodborne illness incident.?
Answer: Yes
Context: classic me : ate some old cheese in the work fridge bc I'm trying to save money and now I have mild food 
poisoning .
Question: Do you think the word old cheese in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: Yes

Now I want you to label the following example:
Context: I cant come into work today because I have food poisoning . I went out with the lads last night and we had
a cheap curry , we all ended up . . .
Question: Do you think the word curry in the given text is a Food entity that is specific food item that caused the
potential foodborne illness incident.?
Answer:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [31]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

Output()

2024-03-14 22:47:58 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:58 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-14 22:47:58 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:59 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-14 22:47:59 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:47:59 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-14 22:47:59 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-14 22:48:00 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report:
              precision    recall  f1-score   support

          No       1.00      0.25      0.40         4
         Yes       0.00      0.00      0.00         0

    accuracy                           0.25         4
   macro avg       0.50      0.12      0.20         4
weighted avg       1.00      0.25      0.40         4

Actual Cost: 0.0043

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.25     │ 4       │ 1.0             │
└──────────┴─────────┴─────────────────┘

In [32]:
metrics = verify_ds.eval()

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ classification_report                                 ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.25     │ 4       │ 1.0             │               precision    recall  f1-score   support │
│          │         │                 │                                                       │
│          │         │                 │           No       1.00      0.25      0.40         4 │
│          │         │                 │          Yes       0.00      0.00      0.00         0 │
│          │         │                 │                                                       │
│          │         │                 │     accuracy                           0.25         4 │
│          │         │                 │    macro avg       0.50      0.12      0.20         4 │
│          │         │                 │ weighted avg       1.00      0.25      0.40         4 │
│          │         │                 │                                                       │
└──────────┴─────────┴─────────────────┴───────────────────────────────────────────────────────┘

In [33]:
new_test_data[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [35]:
verify_ds.df['text_idx'] = new_test_data['text_idx']

In [36]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [37]:
new_result = construct_results(ds.df, new_test_data, f'{label_column}_label', f'{verify_answer_column}_label', "text_idx", pos_column, label_symbol)

In [38]:
metrics = ds.eval()

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ f1     ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.6      │ 10      │ 1.0             │ 0.9264 │
└──────────┴─────────┴─────────────────┴────────┘

In [39]:
new_result.to_csv(output_final_path, index=False)
new_result.to_pickle(output_final_path.replace('.csv', '.pkl'))